In [1]:
#Pray for P100
!nvidia-smi

Tue Nov 26 05:18:19 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install fast-bert

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 317kB 14.2MB/s 
     |████████████████████████████████| 194kB 73.6MB/s 
     |████████████████████████████████| 860kB 52.5MB/s 
     |████████████████████████████████| 645kB 40.0MB/s 
     |████████████████████████████████| 1.0MB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=fab78684b76a7ed8a2526e2cda859befb91e9d7ee2e21126fc70f453a697a3b6
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [3]:
import torch
import torch.nn.functional as F

from transformers import RobertaModel, RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig

#Inference model
config = RobertaConfig.from_pretrained('roberta-large-mnli')
tokenizer = RobertaTokenizer.from_pretrained('roberta-large-mnli')
model = RobertaForSequenceClassification.from_pretrained('roberta-large-mnli')

model.to('cuda') 

100%|██████████| 1425744429/1425744429 [00:36<00:00, 39084252.38B/s]


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm

In [4]:
!pip install gdown
!gdown --id 1WtbXCv3vPB5ql6w0FVDmAEMmWadbrCuG --output dnli.zip
!unzip dnli.zip

Downloading...
From: https://drive.google.com/uc?id=1WtbXCv3vPB5ql6w0FVDmAEMmWadbrCuG
To: /content/dnli.zip
390MB [00:05, 65.7MB/s]
Archive:  dnli.zip
   creating: dnli/
   creating: dnli/dialogue_nli/
  inflating: dnli/dialogue_nli/dialogue_nli_train.jsonl  
  inflating: dnli/dialogue_nli/print_dtypes.py  
  inflating: dnli/dialogue_nli/README.txt  
   creating: __MACOSX/
   creating: __MACOSX/dnli/
   creating: __MACOSX/dnli/dialogue_nli/
  inflating: __MACOSX/dnli/dialogue_nli/._README.txt  
  inflating: dnli/dialogue_nli/dialogue_nli_dev.jsonl  
  inflating: __MACOSX/dnli/dialogue_nli/._dialogue_nli_dev.jsonl  
  inflating: dnli/dialogue_nli/dialogue_nli_test.jsonl  
  inflating: dnli/dialogue_nli/dialogue_nli_verified_test.jsonl  
  inflating: dnli/.DS_Store          
  inflating: __MACOSX/dnli/._.DS_Store  
  inflating: dnli/dialogue_nli.tar.gz  
   creating: dnli/dialogue_nli_extra/
  inflating: dnli/dialogue_nli_extra/dialogue_nli_EXTRA_uu_test.jsonl  
  inflating: dnli/dialogu

In [0]:
#Fine tuning is crucuial for us. because the dataset is very similar: persona based dialogue exchanges
#/ inference format
#/ not sentence based inference, but UTTERANCE based inference. one utterance can consist of
# multiple small sentences (usually max 2)

In [0]:
import json

with open('dnli/dialogue_nli/dialogue_nli_train.jsonl') as f:
    lines = f.readlines()
train_data = json.loads(lines[0])
with open('dnli/dialogue_nli/dialogue_nli_dev.jsonl') as f:
    lines = f.readlines()
val_data = json.loads(lines[0])

In [13]:
label_map = {0:'negative', 1:'neutral', 2:'positive'}
train_data[0]
val_data[0]

{'dtype': 'e2swap_up',
 'id': 'dialogue_nli_train_298308',
 'label': 'negative',
 'sentence1': 'since my dad is a mechanic we had mostly car books .',
 'sentence2': 'my dad is a priest .',
 'triple1': ['my father', 'has_profession', 'mechanic'],
 'triple2': ['my father', 'has_profession', 'priest']}

In [0]:
import pandas as pd
df_train = pd.DataFrame.from_dict(train_data, orient='columns')
df_train['index'] = df_train.index
df_train = df_train[['index','sentence1', 'sentence2','label']]

df_val = pd.DataFrame.from_dict(val_data, orient='columns')
df_val['index'] = df_val.index
df_val = df_val[['index','sentence1', 'sentence2','label']]

In [0]:
df_train.to_csv('train.csv', index=False)
df_val.to_csv('val.csv', index=False)

In [0]:
df.to_csv('temp.csv')

In [0]:
import csv

with open('labels.csv', 'w', newline='') as csvfile:
    fieldnames = ['labels']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writerow({'labels': 'negative'})
    writer.writerow({'labels': 'neutral'})
    writer.writerow({'labels': 'positive'})

In [41]:
from fast_bert.data_cls import BertDataBunch

databunch = BertDataBunch('/content/', '/content/',
                          tokenizer='roberta-large-mnli',
                          model_type='roberta',
                          train_file='train.csv',
                          val_file='val.csv',
                          label_file='labels.csv',
                          text_col=['sentence1','sentence2'],
                          label_col='label',
                          multi_gpu=False,
                          multi_label=False,
                          batch_size_per_gpu=2,
                          max_seq_length=512)

ValueError: ignored

In [32]:
databunch

NameError: ignored